In [1]:
from preprocess import tokenize

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import re
import nltk


In [3]:
data = pd.read_csv("cleaned_data.csv")
data.head(10)

,Input,Labels
0,"in saying goodbye to my son , it ' s like i ' ...","in delivering my son from me , i bury a second..."
1,"and for me , in leaving , mother , i mourn my ...","and i in going , madam , weep o ' er my father..."
2,"the king will be like a husband to you , madam...","you shall find of the king a husband , madam ;..."
3,"if he ' s so good to all his subjects , you ca...",he that so generally is at all times good must...
4,your family has been so important to him that ...,whose worthiness would stir it up where it wan...
5,"since he is generous , there ' s no way he won...",where there is such abundance .
6,how likely is it that his majesty will recover ?,what hope is there of his majesty ' s amendment ?
7,"he ' s given up on his doctors , madam .","he hath abandoned his physicians , madam ;"
8,he ' s wasted a lot of time and hope following...,under whose practises he hath persecuted time ...
9,and all he ' s getting out of it is gradually ...,and finds no other advantage in the process bu...


In [12]:
tokenized_input, tokenized_labels = tokenize(data)

In [13]:
print("Sample Tokenized Input-Labels Pair")
print(tokenized_input[0])
print(tokenized_labels[0])

Sample Tokenized Input-Labels Pair
['in', 'saying', 'goodbye', 'to', 'my', 'son', ',', 'it', "'", 's', 'like', 'i', "'", 'm', 'losing', 'another', 'husband', '.', '<eos>']
['in', 'delivering', 'my', 'son', 'from', 'me', ',', 'i', 'bury', 'a', 'second', 'husband', '.', '<eos>']


In [15]:
print("Sample Input Token Length:", len(tokenized_input[0]))
print("Sample Labels Token Length:", len(tokenized_labels[0]))

Sample Input Token Length: 19
Sample Labels Token Length: 14


In [19]:
tokenized_input_len = [len(line) for line in tokenized_input]
tokenized_labels_len = [len(line) for line in tokenized_labels]

data["Tokenized Input Length"] = tokenized_input_len
data["Tokenized Labels Length"] = tokenized_labels_len

data.head(10)


,Input,Labels,Tokenized Input Length,Tokenized Labels Length
0,"in saying goodbye to my son , it ' s like i ' ...","in delivering my son from me , i bury a second...",19,14
1,"and for me , in leaving , mother , i mourn my ...","and i in going , madam , weep o ' er my father...",45,41
2,"the king will be like a husband to you , madam...","you shall find of the king a husband , madam ;...",22,19
3,"if he ' s so good to all his subjects , you ca...",he that so generally is at all times good must...,24,19
4,your family has been so important to him that ...,whose worthiness would stir it up where it wan...,24,14
5,"since he is generous , there ' s no way he won...",where there is such abundance .,21,7
6,how likely is it that his majesty will recover ?,what hope is there of his majesty ' s amendment ?,11,12
7,"he ' s given up on his doctors , madam .","he hath abandoned his physicians , madam ;",12,9
8,he ' s wasted a lot of time and hope following...,under whose practises he hath persecuted time ...,15,11
9,and all he ' s getting out of it is gradually ...,and finds no other advantage in the process bu...,17,18
